https://www.yuque.com/ol1q37/gi94xp/qgdu4c

In [2]:
import requests,re
import csv
import os

# 构建获取拼音和发音链接的函数

In [10]:
def getpy(wd):
    """获取汉字的拼音和发音链接"""
    head={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36',
          'Cookie': 'BAIDUID=CA2A057597C95C071103B59F29247B15:FG=1; PSTM=1588583905; BIDUPSID=B0376B42F27EF20ACF6127CBBE9D6A5E; COOKIE_SESSION=13_0_9_8_17_20_1_0_9_5_0_7_0_0_0_0_1587444124_0_1588583929%7C9%230_0_1588583929%7C1; BD_UPN=12314753; delPer=0; BD_CK_SAM=1; PSINO=5; H_PS_PSSID=1441_31326_21125_31596_30840_31464_30823_26350_22160; H_PS_645EC=fb1cJLxYMLK42HTwv49rujbfDn0rscUKt9sjk6QdGMjcmYvoJbp3uNN4CzA; BDORZ=B490B5EBF6F3CD402E515D22BCDA1598'}
    url='https://www.baidu.com/s?ie=utf-8&f=8&rsv_bp=1&tn=baidu&wd='  # 找到百度查找的规则，等号后面的即是查询的内容
    r=requests.get(url+wd+'拼音',headers=head)  # 构建百度查找拼音的链接
    r.encoding='utf-8'
    html=r.text
    #找到拼音
    py=re.findall(r'<span class="op_exactqa_detail_word_pronounce">\[([\d\D]*?)\]<',html) 
    #找到发音链接
    pymp3=re.findall(r'<a class="op_exactqa_detail_how_read" data-click="{fm:\'beha\'}" url="([\d\D]*?)" href="javascript:;"',html)
    for i in range(len(py)):
        data.append((wd,py[i],pymp3[i])) #把字，拼音，发音链接保存进列表

# 任务1：测试函数，获取’我‘字的拼音和发音链接

In [15]:
data=[]
getpy('托尔斯泰')
print(data)

[('托尔斯泰', 'tuō ěr sī tài', 'https://ss0.baidu.com/6KAZsjip0QIZ8tyhnq/text2audio?tex=%E6%89%98%28tuo1%29%E5%B0%94%28er3%29%E6%96%AF%28si1%29%E6%B3%B0%28tai4%29&cuid=dict&lan=ZH&ctp=1&pdt=30&vol=9&spd=4')]


# 任务2：

## 逐行读取文件中的汉字，并调用函数获取拼音及发音链接,构建一个CSV文件

In [13]:
def read_hanzi(path):
    """传入文件地址，获取文件中所有汉字的拼音和发音"""
    with open(path) as f:
        for i in f:
            i=i.strip()
            for j in i:
                getpy(j)

In [9]:
def save_csv(save_path):
    """把数据保存为csv文件"""
    with open(save_path,'w',encoding='utf_8_sig',newline='') as f:
        w=csv.writer(f)
        for i in data:
            w.writerow(i)

In [22]:
data = []
read_hanzi('C:\\Users\\Administrator\\Desktop\\汉字.txt')
save_csv('汉字拼音.csv')

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))

# 任务3：存储到数据库

In [32]:
data = []
read_hanzi('汉字2.txt')

In [34]:
data

[('给', 'gěi', 'https://hanyu-word-pinyin-short.cdn.bcebos.com/gei3.mp3'),
 ('给', 'jǐ', 'https://hanyu-word-pinyin-short.cdn.bcebos.com/ji3.mp3'),
 ('定', 'dìng', 'https://hanyu-word-pinyin-short.cdn.bcebos.com/ding4.mp3'),
 ('一', 'yī', 'https://hanyu-word-pinyin-short.cdn.bcebos.com/yi1.mp3'),
 ('个', 'gè', 'https://hanyu-word-pinyin-short.cdn.bcebos.com/ge4.mp3'),
 ('个', 'gě', 'https://hanyu-word-pinyin-short.cdn.bcebos.com/ge3.mp3'),
 ('文', 'wén', 'https://hanyu-word-pinyin-short.cdn.bcebos.com/wen2.mp3'),
 ('本', 'běn', 'https://hanyu-word-pinyin-short.cdn.bcebos.com/ben3.mp3'),
 ('文', 'wén', 'https://hanyu-word-pinyin-short.cdn.bcebos.com/wen2.mp3'),
 ('件', 'jiàn', 'https://hanyu-word-pinyin-short.cdn.bcebos.com/jian4.mp3'),
 ('每', 'měi', 'https://hanyu-word-pinyin-short.cdn.bcebos.com/mei3.mp3'),
 ('行', 'háng', 'https://hanyu-word-pinyin-short.cdn.bcebos.com/hang2.mp3'),
 ('行', 'hàng', 'https://hanyu-word-pinyin-short.cdn.bcebos.com/hang4.mp3'),
 ('行', 'héng', 'https://hanyu-word-pin

# 保存到数据库

In [6]:
from pony.orm import *

db = Database()

class Hz(db.Entity):
    id = PrimaryKey(int, auto=True)
    hz = Optional(str, column='hz')
    py = Optional(str, column='py')
    href = Optional(str, column='href')

In [4]:
def create_database():
    """创建数据库"""
    dbpath = r'hz.sqlite'
    if os.path.exists(dbpath):
        os.remove(dbpath)
    f = open(dbpath, 'w')
    f.close()
            
    db.bind(provider='sqlite', filename=dbpath)
    
    db.generate_mapping(create_tables=True)
    
    set_sql_debug(True)

In [37]:
def save_db():
    """把数据保存进数据库"""
    for i in data:
        s=Hz(hz=i[0],py=i[1],href=i[2])
        db.commit()

In [38]:
create_database()
save_db()

GET NEW CONNECTION
BEGIN IMMEDIATE TRANSACTION
INSERT INTO "Hz" ("hz", "py", "href") VALUES (?, ?, ?)
['给', 'gěi', 'https://hanyu-word-pinyin-short.cdn.bcebos.com/gei3.mp3']

COMMIT
BEGIN IMMEDIATE TRANSACTION
INSERT INTO "Hz" ("hz", "py", "href") VALUES (?, ?, ?)
['给', 'jǐ', 'https://hanyu-word-pinyin-short.cdn.bcebos.com/ji3.mp3']

COMMIT
BEGIN IMMEDIATE TRANSACTION
INSERT INTO "Hz" ("hz", "py", "href") VALUES (?, ?, ?)
['定', 'dìng', 'https://hanyu-word-pinyin-short.cdn.bcebos.com/ding4.mp3']

COMMIT
BEGIN IMMEDIATE TRANSACTION
INSERT INTO "Hz" ("hz", "py", "href") VALUES (?, ?, ?)
['一', 'yī', 'https://hanyu-word-pinyin-short.cdn.bcebos.com/yi1.mp3']

COMMIT
BEGIN IMMEDIATE TRANSACTION
INSERT INTO "Hz" ("hz", "py", "href") VALUES (?, ?, ?)
['个', 'gè', 'https://hanyu-word-pinyin-short.cdn.bcebos.com/ge4.mp3']

COMMIT
BEGIN IMMEDIATE TRANSACTION
INSERT INTO "Hz" ("hz", "py", "href") VALUES (?, ?, ?)
['个', 'gě', 'https://hanyu-word-pinyin-short.cdn.bcebos.com/ge3.mp3']

COMMIT
BEGIN IMM

['发', 'fà', 'https://hanyu-word-pinyin-short.cdn.bcebos.com/fa4.mp3']

COMMIT
BEGIN IMMEDIATE TRANSACTION
INSERT INTO "Hz" ("hz", "py", "href") VALUES (?, ?, ?)
['音', 'yīn', 'https://hanyu-word-pinyin-short.cdn.bcebos.com/yin1.mp3']

COMMIT
BEGIN IMMEDIATE TRANSACTION
INSERT INTO "Hz" ("hz", "py", "href") VALUES (?, ?, ?)
['链', 'liàn', 'https://hanyu-word-pinyin-short.cdn.bcebos.com/lian4.mp3']

COMMIT
BEGIN IMMEDIATE TRANSACTION
INSERT INTO "Hz" ("hz", "py", "href") VALUES (?, ?, ?)
['接', 'jiē', 'https://hanyu-word-pinyin-short.cdn.bcebos.com/jie1.mp3']

COMMIT
BEGIN IMMEDIATE TRANSACTION
INSERT INTO "Hz" ("hz", "py", "href") VALUES (?, ?, ?)
['存', 'cún', 'https://hanyu-word-pinyin-short.cdn.bcebos.com/cun2.mp3']

COMMIT
BEGIN IMMEDIATE TRANSACTION
INSERT INTO "Hz" ("hz", "py", "href") VALUES (?, ?, ?)
['储', 'chǔ', 'https://hanyu-word-pinyin-short.cdn.bcebos.com/chu3.mp3']

COMMIT
BEGIN IMMEDIATE TRANSACTION
INSERT INTO "Hz" ("hz", "py", "href") VALUES (?, ?, ?)
['到', 'dào', 'https://

In [8]:
create_database()

ValueError: When in interactive mode, please provide absolute file path. Got: 'hz.sqlite'